### 공간정보 데이터 시각화 


####함수정의

In [ ]:
# GPS좌표 형식 변환 함수 (KATEC -> WGS804) 
def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y)

####격자별 사건의 빈도수

In [ ]:
# 사건별 좌표
data_DF['TOTAL_CNT'] = data_DF.loc[:,'MCHN_ACDNT_OCRN_CNT':'FALLING_OCRN_CNT'].sum(axis=1)
total_DF = data_DF[['GRID_ID', 'GRID_X_AXIS', 'GRID_Y_AXIS','TOTAL_CNT']].groupby(['GRID_ID', 'GRID_X_AXIS', 'GRID_Y_AXIS']).sum().reset_index()

In [ ]:
# 사건 빈도수
data_DF['TOTAL_CNT'].value_counts()

0.0    150858
1.0      2373
2.0       184
3.0        10
Name: TOTAL_CNT, dtype: int64

In [ ]:
# 격자별 사건 빈도수 
pd.DataFrame(total_DF["TOTAL_CNT"].value_counts()).T

,0.0,1.0,2.0,3.0,4.0,5.0,8.0,12.0,6.0,10.0,...,98.0,61.0,49.0,28.0,91.0,71.0,54.0,105.0,44.0,100.0
TOTAL_CNT,238,71,37,23,16,9,9,6,5,5,...,1,1,1,1,1,1,1,1,1,1


모든 건수격자지도 시각화

In [ ]:
def f_color(x):
    if x > 70:        # 70건이상 빨강색
        a = '#FF4F00'     
    elif x > 30:      # 30건이상 주황색   
        a = '#FCB100' 
    elif x > 10:      # 10건이상 노란색
        a = '#E0F500' 
    elif x > 1:       # 1건이상 연두색
        a = '#8CF700'  
    elif x >= 0:      # 0건  회색
        a = '#C6C6C6'
    else :            # 유동인구 0이며 사건이 0건인 경우 색 X
        pass 
    return a

In [ ]:
# 총 사건 빈도수 지도화 확인 
total_DF['cell_color']= total_DF['TOTAL_CNT'].apply(lambda x : f_color(x))
geodf = total_DF.rename(columns = {'GRID_ID' : 'id','GRID_X_AXIS':'x', 'GRID_Y_AXIS':'y', 'TOTAL_CNT':'total'})
geodf = geodf[['id','x','y','cell_color','total']]

# 원주 격자 좌표화 
cell_size = 1000
a = cell_size//2
geodf['nwx'], geodf['nwy'] = geodf['x']-a, geodf['y']+a
geodf['nex'], geodf['ney'] = geodf['x']+a, geodf['y']+a
geodf['swx'], geodf['swy'] = geodf['x']-a, geodf['y']-a
geodf['sex'], geodf['sey'] = geodf['x']+a ,geodf['y']-a

geodf['lng'], geodf['lat'] = katec_to_wgs84(geodf.x.to_list(), geodf.y.to_list())
geodf['nwlng'], geodf['nwlat'] = katec_to_wgs84(geodf.nwx.to_list(), geodf.nwy.to_list())
geodf['nelng'], geodf['nelat'] = katec_to_wgs84(geodf.nex.to_list(), geodf.ney.to_list())
geodf['swlng'], geodf['swlat'] = katec_to_wgs84(geodf.swx.to_list(), geodf.swy.to_list())
geodf['selng'], geodf['selat'] = katec_to_wgs84(geodf.sex.to_list(), geodf.sey.to_list())

map = folium.Map(location=[geodf['lat'].mean(), geodf['lng'].mean()], zoom_start=11, tiles="OpenStreetMap")

geodf.apply(lambda x : folium.Polygon(locations=[[x.nwlat, x.nwlng], [x.swlat,x.swlng], [x.selat,x.selng], [x.nelat,x.nelng]], 
                                        color='white', 
                                        popup = 'id:'+ str(x.id),
                                        weight=0.1,
                                        fill=True,
                                        fill_color=x.cell_color,
                                        fill_opacity=0.6
                                        ).add_to(map), axis =1)

# 주로 사건이 발생되는 장소 : 원주 기업도시 / 원주 도심지역 / 문막읍 / 흥업리 / 신림면 / 태장농공단지
folium.Marker(location=[37.376218, 127.867677], popup="원주 기업도시", icon=folium.Icon(color="red", icon="info-sign"),).add_to(map)
folium.Marker(location=[37.340229, 127.935455], popup="원주 도심지역", icon=folium.Icon(color="blue", icon="info-sign"),).add_to(map)
folium.Marker(location=[37.313082, 127.818659], popup="문막읍", icon=folium.Icon(color="green", icon="info-sign"),).add_to(map)
folium.Marker(location=[37.30817, 127.918924], popup="흥업리", icon=folium.Icon(color="black", icon="info-sign"),).add_to(map)
folium.Marker(location=[37.232082, 128.082054], popup="신림면", icon=folium.Icon(color="pink", icon="info-sign"),).add_to(map)
folium.Marker(location=[37.403312, 127.946698], popup="태장농공단지", icon=folium.Icon(color="orange", icon="info-sign"),).add_to(map)

map